# Machine Learning

## 1. Initial Setup

- Set variables and hyperparameters
- Import libraries
- Load dataset

In [ ]:
# Max allowable epochs for each model type
MAX_EPOCHS = 2000

# Hyperparameters for Hyperparameter Optimization using Optuna
OPTUNA_N_JOBS = 8                   # Number of parallel jobs for hyperparameter optimization - controls how many trials run simultaneously
OPTUNA_N_TRIALS = 40                # Total number of optimization trials to run - more trials generally lead to better hyperparameter discovery
OPTUNA_PRUNE_N_STARTUP_TRIALS = 8   # Number of random trials before pruning starts - ensures diverse exploration before early stopping
OPTUNA_PRUNE_WARMUP_STEPS = 8       # Number of steps to wait before pruning can occur - prevents premature termination of promising trials

In [ ]:
PROCESSED_PATH = "../data/ANA HIDROWEB/RIO MEIA PONTE/processed.csv"

In [ ]:
# Python native libraries
from typing import List, Dict, Any, Tuple
import os
import warnings
import uuid

# Data manipulation and preprocessing
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# PyTorch libraries and derivatives
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import lightning as L
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
from torchinfo import summary

# Optuna Libraries
import optuna
from optuna.integration import PyTorchLightningPruningCallback

# Set precision for matrix multiplication in order to optimize performance
torch.set_float32_matmul_precision("high")

# Set random seed for reproducibility
L.seed_everything(42)

In [ ]:
df = pd.read_csv(
    PROCESSED_PATH,
    sep=";",
    parse_dates=["date"],
    dayfirst=True,
)

df.set_index("date", inplace=True)
df.index = pd.to_datetime(df.index)

df.head()

## 2. Data Preprocessing

- Conferir ausência de dados nulos
- Retirar as colunas de vazão, uma vez que está altamente correlacionada à coluna de nível
- Normalização dos dados

In [ ]:
# Count missing values in each column
def print_missing_values(data):
    missing_values = data.isnull().sum()
    print("Missing values in each column:")
    print(missing_values[missing_values > 0])

print_missing_values(df)

In [ ]:
# Remove columns containing 'flow' from the dataframe
df = df.loc[:, ~df.columns.str.contains('flow')]
df.head()

In [ ]:
scaler = StandardScaler()
df_normalized = pd.DataFrame(
    scaler.fit_transform(df),
    index=df.index,
    columns=df.columns
)
df_normalized.head()

## 3. Criação dos Datasets

- Criar dataset para série temporal
- Dividir o dataset em treino, validação e teste

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length=24, target_col="level_downstream_max"):
        self.data = data.reset_index(drop=True)
        self.sequence_length = sequence_length
        self.target_col = self.data.columns.get_loc(target_col)

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        x = self.data.iloc[idx : idx + self.sequence_length].values  # (sequence_length, num_features)
        y = self.data.iloc[idx + self.sequence_length, self.target_col] # scalar value
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


class NonOverlappingConcatDataset(Dataset):
    def __init__(self, datasets):
        self.datasets = datasets
        self.cumulative_lengths = []
        total = 0
        for d in datasets:
            self.cumulative_lengths.append(total)
            total += len(d)
        self.total_length = total

    def __len__(self):
        return self.total_length

    def __getitem__(self, idx):
        # Find which dataset this idx belongs to
        for i in range(len(self.datasets)):
            if idx < self.cumulative_lengths[i] + len(self.datasets[i]):
                local_idx = idx - self.cumulative_lengths[i]
                return self.datasets[i][local_idx]
        raise IndexError("Index out of range")


def split_train_validation_test(
    data: pd.DataFrame, train_size: float = 0.8, val_size: float = 0.1
):
    train_end = int(len(data) * train_size)
    val_end = int(len(data) * (train_size + val_size))

    train_data = data.iloc[:train_end]
    val_data = data.iloc[train_end:val_end]
    test_data = data.iloc[val_end:]

    return train_data, val_data, test_data


def create_datasets(
    sequence_length: int,
) -> Tuple[
    NonOverlappingConcatDataset,
    NonOverlappingConcatDataset,
    NonOverlappingConcatDataset,
]:
    datasets_train: List[TimeSeriesDataset] = []
    datasets_validation: List[TimeSeriesDataset] = []
    datasets_test: List[TimeSeriesDataset] = []
    for year in range(2010, 2024):
        year_data = df_normalized[df_normalized.index.year == year].reset_index(
            drop=True
        )
        if not year_data.empty:
            train_data, validation_data, test_data = split_train_validation_test(
                year_data
            )
            datasets_train.append(
                TimeSeriesDataset(train_data, sequence_length=sequence_length)
            )
            datasets_validation.append(
                TimeSeriesDataset(validation_data, sequence_length=sequence_length)
            )
            datasets_test.append(
                TimeSeriesDataset(test_data, sequence_length=sequence_length)
            )

    train_dataset = NonOverlappingConcatDataset(datasets_train)
    validation_dataset = NonOverlappingConcatDataset(datasets_validation)
    test_dataset = NonOverlappingConcatDataset(datasets_test)

    return train_dataset, validation_dataset, test_dataset

# Debugging example
# fake_train, fake_validation, fake_test = create_datasets(
#     sequence_length=2,
# )  # Example with sequence length of 2

# print(fake_train[0][1])
# fake_train[0][0].shape, fake_train[0][1].shape  # Should be (2, num_features), scalar

In [ ]:
# Test for TimeSeriesDataset

# Create a simple DataFrame with increasing integers
test_df = pd.DataFrame({'A': np.arange(10)})

# Window length = 3
ts_dataset = TimeSeriesDataset(test_df, sequence_length=3, target_col='A')

print("Testing TimeSeriesDataset:")
for i in range(len(ts_dataset)):
    x, y = ts_dataset[i]
    print(f"Index {i}: x = {x.squeeze().numpy()}, y = {y.numpy()}")

# Test for NonOverlappingConcatDataset
# Create two small TimeSeriesDatasets
df1 = pd.DataFrame({'A': np.arange(5)})
df2 = pd.DataFrame({'A': np.arange(10, 15)})

ds1 = TimeSeriesDataset(df1, sequence_length=2, target_col='A')
ds2 = TimeSeriesDataset(df2, sequence_length=2, target_col='A')

concat_ds = NonOverlappingConcatDataset([ds1, ds2])

print("\nTesting NonOverlappingConcatDataset")
print("Should not overlap and should concatenate correctly:")
for i in range(len(concat_ds)):
    x, y = concat_ds[i]
    print(f"Index {i}: x = {x.squeeze().numpy()}, y = {y.numpy()}")


## 4. Definição dos Modelos

In [ ]:
class MLP(L.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_layers,
        learning_rate: float = 0.0001,
    ):
        super(MLP, self).__init__()
        self.save_hyperparameters()

        # MLP para cada passo temporal
        layers = []
        in_features = input_size
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(in_features, hidden_size))
            layers.append(nn.ReLU())
            in_features = hidden_size
        self.step_mlp = nn.Sequential(*layers)
        self.final_layer = nn.Linear(in_features, 1)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        # x: (batch, window_length, input_size)
        # Aplica o MLP em cada passo temporal    print("x type:", type(x))
        batch, window, features = x.shape
        x = x.view(-1, features)  # (batch * window_length, input_size)
        out = self.step_mlp(x)  # (batch * window_length, hidden)
        out = out.view(batch, window, -1)  # (batch, window_length, hidden)
        out = out.mean(dim=1)  # (batch, hidden) - agregação temporal
        out = self.final_layer(out)  # (batch, input_size)
        return out.squeeze(-1)  # (batch,)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss


class LSTM(L.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        learning_rate: float = 0.0001,
    ):
        super(LSTM, self).__init__()
        self.save_hyperparameters()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        out, _ = self.lstm(x)  # (batch, sequence_length, hidden_size)
        return self.fc(out[:, -1, :]).squeeze(-1)  # (batch,)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss


class GRU(L.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        learning_rate: float = 0.0001,
    ):
        super(GRU, self).__init__()
        self.save_hyperparameters()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :]).squeeze(-1)  # (batch,)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss


print(summary(MLP(input_size=5, hidden_layers=[10, 10])))
print("\n")
print(summary(LSTM(input_size=5, hidden_size=10, num_layers=2)))
print("\n")
print(summary(GRU(input_size=5, hidden_size=10, num_layers=2)))

## 5. Treinamento dos Modelos

### 5.1. training functions

In [ ]:
def train_model(
    model,
    train_data,
    val_data,
    logger,
    epochs,
    batch_size,
    callbacks=[],
    verbose=True,
) -> None:
    """
    Treina o modelo com os dados de treino e validação.
    """

    train_loader = DataLoader(
        train_data, batch_size=batch_size, shuffle=True, drop_last=True
    )
    validation_loader = DataLoader(
        val_data, batch_size=4, shuffle=False, drop_last=True
    )

    trainer = Trainer(
        max_epochs=epochs,
        accelerator="auto",
        devices=1,
        logger=logger,
        enable_progress_bar=verbose,
        log_every_n_steps=1,
        callbacks=callbacks,
        deterministic=True,
        check_val_every_n_epoch=1,
        enable_model_summary=verbose,
    )
    trainer.fit(model, train_loader, validation_loader)


def evaluate_model(model, test_data, logger, batch_size=4):
    """
    Avalia o modelo com os dados de teste.
    """
    print("\nEvaluating model. Using the test dataset.")
    test_loader = DataLoader(
        test_data, batch_size=batch_size, shuffle=False, drop_last=True
    )

    trainer = Trainer(
        accelerator="auto",
        devices=1,
        logger=logger,
        enable_progress_bar=True,
        deterministic=True,
    )
    trainer.test(model, test_loader)


def run_experiment(
    model_class: type[L.LightningModule],
    name: str,
    model_kwargs: Dict[str, Any] = {},
    train_kwargs: Dict[str, Any] = {},
    evaluate: bool = True,
    extra_callbacks: List[L.Callback] = [],
    verbose: bool = True,
    id: str = "",
):
    """
    Executa um experimento de treinamento e avaliação do modelo.
    """
    # Logger
    logger = TensorBoardLogger(save_dir=os.getcwd(), name=f"lightning_logs/{name}")

    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss", mode="min", save_top_k=1, filename=f"best_model_{id}"
    )

    early_stopping_callback = EarlyStopping(
        monitor="val_loss",
        min_delta=1e-4,
        patience=10,
        mode="min",
        strict=False,
    )

    train_dataset, validation_dataset, test_dataset = create_datasets(
        sequence_length=train_kwargs.get("sequence_length", 5)
    )

    # Remove 'sequence_length' from train_kwargs if present
    train_kwargs = dict(train_kwargs)  # make a copy to avoid side effects
    train_kwargs.pop("sequence_length", None)

    input_size = train_dataset[0][0].shape[1]  # Assuming all inputs have the same shape

    model = model_class(input_size=input_size, **(model_kwargs or {}))

    train_model(
        model,
        train_dataset,
        validation_dataset,
        logger=logger,
        **(train_kwargs or {}),
        callbacks=[checkpoint_callback, early_stopping_callback, *extra_callbacks],
        verbose=verbose,
    )

    best_validation_loss = checkpoint_callback.best_model_score
    best_model_path = checkpoint_callback.best_model_path

    # To view the logs, you can use TensorBoard.
    # If 'logs' is a SummaryWriter, get its log_dir and launch TensorBoard:
    print(f"To view logs, run in terminal:\n  tensorboard --logdir {logger.log_dir}")
    print("Best model saved at:", best_model_path)

    if evaluate:
        best_model = model_class.load_from_checkpoint(best_model_path, **model_kwargs)
        evaluate_model(best_model, test_dataset, logger, batch_size=4)

    return {
        "best_validation_loss": (
            best_validation_loss.item() if best_validation_loss is not None else None
        ),
        "best_model_path": best_model_path,
    }

### 5.2. Hyperparameter optimization functions

In [ ]:
def get_optimization_args(model_class: L.LightningModule, trial):
    """
    Retorna os argumentos necessários para executar o experimento.
    """
    if model_class == MLP:
        model_kwargs = {
            "hidden_layers": trial.suggest_categorical(
                "hidden_layers",
                (
                    [32, 16],
                    [48, 24],
                    [64, 32],
                    [96, 48],
                    [128, 64],
                    [64, 32, 16],
                    [128, 64, 32],
                ),
            ),
            "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),
        }
        train_kwargs = {
            "epochs": MAX_EPOCHS,
            "batch_size": trial.suggest_int("batch_size", 2, 32),
            "sequence_length": trial.suggest_int("sequence_length", 2, 15),
        }
    elif model_class == LSTM:
        model_kwargs = {
            "hidden_size": trial.suggest_int("hidden_size", 16, 128),
            "num_layers": trial.suggest_int("num_layers", 1, 3),
            "learning_rate": trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True),
        }
        train_kwargs = {
            "epochs": MAX_EPOCHS,
            "batch_size": trial.suggest_int("batch_size", 2, 32),
            "sequence_length": trial.suggest_int("sequence_length", 2, 15),
        }
    elif model_class == GRU:
        model_kwargs = {
            "hidden_size": trial.suggest_int("hidden_size", 16, 128),
            "num_layers": trial.suggest_int("num_layers", 1, 3),
            "learning_rate": trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True),
        }
        train_kwargs = {
            "epochs": MAX_EPOCHS,
            "batch_size": trial.suggest_int("batch_size", 8, 32),
            "sequence_length": trial.suggest_int("sequence_length", 3, 12),
        }
    else:
        raise ValueError(f"Unsupported model class: {model_class}")
    return model_kwargs, train_kwargs


def get_objective_fn(model_class, name):
    """
    Retorna uma função de objetivo para otimização de hiperparâmetros.
    """

    def objective(trial: optuna.trial.Trial) -> float:
        # Ajusta os hiperparâmetros com base no trial
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning)
            model_kwargs, train_kwargs = get_optimization_args(model_class, trial)

        optuna_pruning_callback = PyTorchLightningPruningCallback(
            trial, monitor="val_loss"
        )

        results = run_experiment(
            model_class=model_class,
            name=name,
            model_kwargs=model_kwargs,
            train_kwargs=train_kwargs,
            evaluate=False,
            extra_callbacks=[optuna_pruning_callback],
            verbose=False,
            id=uuid.uuid4().hex[:8],
        )
        return results["best_validation_loss"]

    return objective


def run_optimization(objective_fn, n_trials=OPTUNA_N_TRIALS) -> optuna.study.Study:
    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=OPTUNA_PRUNE_N_STARTUP_TRIALS,
        n_warmup_steps=OPTUNA_PRUNE_WARMUP_STEPS,
    )
    study = optuna.create_study(direction="minimize", pruner=pruner)
    study.optimize(
        objective_fn, n_trials=n_trials, n_jobs=OPTUNA_N_JOBS, show_progress_bar=True
    )

    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    return study

### 5.3. Training loop

In [ ]:
mlp_objective_fn = get_objective_fn(
    MLP,
    "mlp",
)

study_mlp = run_optimization(mlp_objective_fn, n_trials=OPTUNA_N_TRIALS)

mlp_optimized_results = run_experiment(
    MLP,
    "mlp_optimized",
    model_kwargs={
        "hidden_layers": study_mlp.best_trial.params["hidden_layers"],
        "learning_rate": study_mlp.best_trial.params["learning_rate"],
    },
    train_kwargs={
        "epochs": MAX_EPOCHS,
        "batch_size": study_mlp.best_trial.params["batch_size"],
        "sequence_length": study_mlp.best_trial.params["sequence_length"],
    },
    evaluate=True,
    verbose=True,
)

print("\nMLP Optimized Results:", mlp_optimized_results)

In [ ]:
lstm_objective_fn = get_objective_fn(
    LSTM,
    "lstm",
)
study_lstm = run_optimization(lstm_objective_fn, n_trials=OPTUNA_N_TRIALS)
lstm_optimized_results = run_experiment(
    LSTM,
    "lstm_optimized",
    model_kwargs={
        "hidden_size": study_lstm.best_trial.params["hidden_size"],
        "num_layers": study_lstm.best_trial.params["num_layers"],
        "learning_rate": study_lstm.best_trial.params["learning_rate"],
    },
    train_kwargs={
        "epochs": MAX_EPOCHS,
        "batch_size": study_lstm.best_trial.params["batch_size"],
    },
    evaluate=True,
    verbose=True,
)
print("\nLSTM Optimized Results:", lstm_optimized_results)

In [ ]:
gru_objective_fn = get_objective_fn(
    GRU,
    "gru",
)
study_gru = run_optimization(gru_objective_fn, n_trials=OPTUNA_N_TRIALS)
gru_optimized_results = run_experiment(
    GRU,
    "gru_optimized",
    model_kwargs={
        "hidden_size": study_gru.best_trial.params["hidden_size"],
        "num_layers": study_gru.best_trial.params["num_layers"],
        "learning_rate": study_gru.best_trial.params["learning_rate"],
    },
    train_kwargs={
        "epochs": MAX_EPOCHS,
        "batch_size": study_gru.best_trial.params["batch_size"],
    },
    evaluate=True,
    verbose=True,
)
print("\nGRU Optimized Results:", gru_optimized_results)